In [1]:
import numpy as np
import argparse
import os
import sys
import random
import torch
import torch.fft as F
from importlib import reload
from torch.nn.functional import relu
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
import utils,mask_backward_new
import matplotlib.pyplot as plt
# from maskbackward import mask_backward
from mask_backward_new import mask_backward, mask_eval
from utils import mask_complete , mask_makebinary, kplot, mask_filter, raw_normalize,\
                    get_x_f_from_yfull, mask_naiveRand, apply_mask, sigmoid_binarize, compute_hfen

sys.path.insert(0,'/home/huangz78/mri/unet/')
from unet_model import UNet

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [2]:
np.linspace(-5.5,-3.5,11)

array([-5.5, -5.3, -5.1, -4.9, -4.7, -4.5, -4.3, -4.1, -3.9, -3.7, -3.5])

#### import data to test mask_backward function

In [ ]:
# data_gt = np.load('/home/huangz78/data/data_gt.npz')
# datafornn = np.load('/home/huangz78/data/datafornn.npz')
data = np.load('/home/huangz78/data/traindata_x.npz')
dtyp = torch.float
xfull = torch.tensor(data['xfull'],dtype=dtyp)
fullmask = torch.tensor(data['mask'])

##### greedy mask and relevant data

In [2]:
from sklearn.model_selection import train_test_split

imgs   = torch.tensor( np.load('/home/huangz78/data/data_gt.npz')['imgdata'] ).permute(2,0,1)
masks  = torch.tensor( np.load('/home/huangz78/data/data_gt_greedymask.npz')['mask'].T ) # labels are already rolled
imgNum = imgs.shape[0]
traininds, testinds = train_test_split(np.arange(imgNum),random_state=0,shuffle=True,train_size=round(imgNum*0.8))

In [3]:
### load a mnet
from mnet import MNet
# mnet = MNet(out_size=320-24)
mnet = MNet(beta=1,in_channels=2,out_size=320-24, imgsize=(320,320),poolk=3)
mnetpath = '/home/huangz78/checkpoints/mnet.pth'
checkpoint = torch.load(mnetpath)
mnet.load_state_dict(checkpoint['model_state_dict'])
print('MNet loaded successfully from: ' + mnetpath)
mnet.eval()

### load a unet for maskbackward
UNET = UNet(n_channels=1,n_classes=1,bilinear=True,skip=False)
# unetpath = '/home/huangz78/checkpoints/unet_'+ str(UNET.n_channels) +'.pth'
unetpath = '/home/huangz78/checkpoints/unet_1_False.pth'

# UNET = UNet(n_channels=1,n_classes=1,bilinear=False,skip=True)
# unetpath = '/home/huangz78/checkpoints/unet_1_True.pth'
checkpoint = torch.load(unetpath)
UNET.load_state_dict(checkpoint['model_state_dict'])
print('Unet loaded successfully from: ' + unetpath )
UNET.train()
print('nn\'s are ready')

MNet loaded successfully from: /home/huangz78/checkpoints/mnet.pth
Unet loaded successfully from: /home/huangz78/checkpoints/unet_1_False.pth
nn's are ready


In [ ]:
# reload(utils)
# from utils import hfen
reload(mask_backward_new)
from mask_backward_new import mask_backward, mask_eval

In [4]:
# batchsize = 5
# xstar = xfull[0:batchsize,:,:]
xstar    = imgs[testinds[testinds.size//2:],:,:]
for ind in range(xstar.shape[0]):
    xstar[ind,:,:] = xstar[ind,:,:]/xstar[ind,:,:].max()
full_gredmask = masks[testinds[testinds.size//2:],:]
corefreq = 24
budget   = 56
yfull = torch.fft.fftshift(F.fftn(xstar,dim=(1,2),norm='ortho')).to(torch.cfloat) # y is ROLLED!
lowfreqmask,_,_ = mask_naiveRand(xstar.shape[1],fix=corefreq,other=0,roll=True)

z = apply_mask(lowfreqmask,yfull,mode='r')
highmask = sigmoid_binarize(raw_normalize(mnet(z),budget=budget))
randmask = torch.zeros(highmask.shape)
for ind in range(highmask.shape[0]):
    sampinds = np.random.choice(highmask.shape[1],int(highmask[ind,:].sum()),replace=False)
    randmask[ind,sampinds] = 1
lowfmask,_,_ = mask_naiveRand(xstar.shape[1]-corefreq,fix=torch.sum(highmask[0,:]),other=0,roll=True)
lowfmask = lowfmask.repeat(highmask.shape[0],1)
# x_lf     = get_x_f_from_yfull(lowfreqmask,yfull)
# highmask = sigmoid_binarize(mnet(x_lf.view(batchsize,1,xstar.shape[1],xstar.shape[2])))

NN         = 9
alpha_grid = 10**(np.linspace(-5.5,-3,NN))
c_grid     = np.array([1e-4,1e-3,1e-2,1e-1,1e0])
l2loss   = np.zeros((NN,5))
hfen = np.zeros((NN,5))
sparsity = np.zeros((NN,5))
########################################  
## (1) mask_backward
########################################    
maxIter_mb = 15
lr_mb      = 1e-2

# alpha = 1e-5
# c = 1e-2

In [ ]:
mask_backward(highmask,xstar,unet=UNET, mnet=mnet,\
                          beta=1.,alpha=1e-4,c=1e-2,\
                          maxIter=maxIter_mb+10,seed=0,break_limit=np.inf,\
                          lr=lr_mb,mode='UNET',budget=budget,normalize=False,\
                          verbose=True,dtyp=torch.float,testmode='sigpy',hfen=True)

In [ ]:
c_ind = 0
for c in c_grid:
    print(f'c_ind {c_ind+1} out of {len(c_grid)}')
    a_ind = 0
    for alpha in alpha_grid:
        print(f'alpha_ind {a_ind+1} out of {len(alpha_grid)}')
        # load a unet for maskbackward
        UNET = UNet(n_channels=1,n_classes=1,bilinear=True,skip=False)
        unetpath = '/home/huangz78/checkpoints/unet_1_False.pth'
        checkpoint = torch.load(unetpath)
        UNET.load_state_dict(checkpoint['model_state_dict'])
        UNET.train()
    # highmask_refined,unet = mask_backward(highmask,xstar,unet=UNET, mnet=mnet,\
    #                   beta=1.,alpha=alpha,c=c,\
    #                   maxIter=maxIter_mb,seed=0,break_limit=maxIter_mb*3//5,\
    #                   lr=lr_mb,mode='UNET',budget=budget,normalize=False,\
    #                   verbose=True,dtyp=torch.float)
        (l2loss[a_ind,c_ind],hfen[a_ind,c_ind]),sparsity[a_ind,c_ind] =\
                        mask_backward(highmask,xstar,unet=UNET, mnet=mnet,\
                          beta=1.,alpha=alpha,c=c,\
                          maxIter=maxIter_mb,seed=0,break_limit=np.inf,\
                          lr=lr_mb,mode='UNET',budget=budget,normalize=False,\
                          dtyp=torch.float,verbose=True,testmode='sigpy',hfen=True)
        a_ind += 1
    print('\n')
    c_ind += 1

c_ind 1 out of 5
alpha_ind 1 out of 9


/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/torch/autograd/__init__.py:147: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554786529/work/aten/src/ATen/native/Copy.cpp:219.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


Iter 10, rows added: 0.1, rows reducted: 0.0
Iter 11, rows added: 0.7, rows reducted: 0.1
Iter 12, rows added: 12.65, rows reducted: 0.0
Iter 13, rows added: 130.2, rows reducted: 0.05
Iter 14, rows added: 78.55, rows reducted: 0.0
Iter 15, rows added: 7.45, rows reducted: 0.0

return at Iter ind:  15
loss of returned mask: (0.00915240696204989, 0.0496024175095589)
samp. ratio: 0.9671875, Recon. rel. err: 0.060508316646377694 

alpha_ind 2 out of 9
Iter 11, rows added: 0.7, rows reducted: 0.0
Iter 12, rows added: 14.2, rows reducted: 0.0
Iter 13, rows added: 60.45, rows reducted: 0.0
Iter 14, rows added: 105.1, rows reducted: 7.85
Iter 15, rows added: 32.75, rows reducted: 0.15

return at Iter ind:  15
loss of returned mask: (0.018273703625847623, 0.09272527920854942)
samp. ratio: 0.89125, Recon. rel. err: 0.07882034908293895 

alpha_ind 3 out of 9
Iter 12, rows added: 1.9, rows reducted: 0.0
Iter 13, rows added: 19.3, rows reducted: 0.0
Iter 14, rows added: 141.45, rows reducted: 0.0


In [ ]:
full_highmask = mask_complete(highmask,320)
full_randmask = mask_complete(randmask,320)
full_lowfmask = mask_complete(lowfmask,320)

In [ ]:
# mnet_loss = mask_eval(full_highmask,xstar,mode='UNET',UNET=UNET)
# rand_loss = mask_eval(full_randmask,xstar,mode='UNET',UNET=UNET)
# lowf_loss = mask_eval(full_lowfmask,xstar,mode='UNET',UNET=UNET)
# print('mode = UNet')
mnet_loss = mask_eval(full_highmask,xstar,mode='sigpy')
rand_loss = mask_eval(full_randmask,xstar,mode='sigpy')
lowf_loss = mask_eval(full_lowfmask,xstar,mode='sigpy')
lowg_loss = mask_eval(full_gredmask,xstar,mode='sigpy')
print('mode = sigpy')
print(mnet_loss)
print(rand_loss)
print(lowf_loss)
print(lowg_loss)

In [ ]:
plt.figure()
plt.plot(alpha_grid,maskloss)
plt.title('mask loss')
plt.xscale('log')

plt.figure()
plt.plot(alpha_grid,sparsity,label='actual')
plt.plot(alpha_grid,(corefreq+budget)/320*np.ones(alpha_grid.shape),label='target')
plt.title('mask sparsity')
plt.xscale('log')

In [ ]:
markers = ['H', 'D', 'P', 'X','+']
plt.figure(figsize=(10,8))
for c_ind in range(len(c_grid)):
    plt.plot(alpha_grid,maskloss[:,c_ind],label='c='+str(c_grid[c_ind]),marker=markers[c_ind],markersize=7)
plt.xlabel('alpha')
plt.title('mask loss')
plt.xscale('log')
plt.legend(loc='best')

plt.figure(figsize=(10,8))
for c_ind in range(len(c_grid)):
    plt.plot(alpha_grid,sparsity[:,c_ind],label='c='+str(c_grid[c_ind]),marker=markers[c_ind],markersize=7)
plt.plot(alpha_grid,(corefreq+budget)/320*np.ones(alpha_grid.shape),'-.',label='target')
plt.title('mask sparsity')
plt.xscale('log')
plt.legend(loc='best')
plt.xlabel('alpha')

In [ ]:
(highmask_refined - highmask).abs().sum()

ind = 0
highmask_refined[ind,:] - highmask[ind,:]

### old tests

In [ ]:
data_gt = np.load('/home/huangz78/data/data_gt.npz')
picind = np.random.randint(199)
xstar = data_gt['imgdata'][:,:,27]
xstar = xstar/np.max(np.abs(xstar))
# highmask = datafornn['labels'][picind,:]
plt.figure()
plt.imshow(xstar)
plt.colorbar()
plt.show()

In [ ]:
# reload(maskbackward)
# from maskbackward import mask_backward

#mode UNET:

reload(mask_backward_new)
from mask_backward_new import mask_backward,mask_eval

dtyp = torch.float
# unroll_block = 8; Lambda=6.1e-4; rho=1e1

# base = .05; expand = .15
# highmask = torch.zeros((round(320*(1-base))),dtype=torch.double)
# highmask[np.random.choice(round(320*(1-base)),round(320*expand),replace=False)] = 1

highmask = mask_filter(fullmask,base=round(320*0.05),roll=True)
print(highmask.shape)

In [ ]:
n_channels = 1
UNET =  UNet(n_channels=n_channels,n_classes=n_channels,bilinear=True,skip=False)
checkpoint = torch.load('/home/huangz78/checkpoints/unet_' + str(n_channels) + '.pth')
UNET.load_state_dict(checkpoint['model_state_dict'])
UNET.eval()

In [ ]:
## quick comparison between ifft recon and unet recon
imgHeg = 320; imgWid = 320
fullmask = torch.tensor( mask_complete(highmask,imgHeg,dtyp=torch.float) )
kplot(fullmask)
print('sparsity of fullmask = ',fullmask.sum().item()/imgHeg)

In [ ]:
xstar = torch.tensor(xstar,dtype=dtyp)
DTyp = torch.cfloat if dtyp==torch.float else torch.cdouble
y = torch.fft.fftshift(F.fftn(xstar,dim=(0,1),norm='ortho'))
z = torch.fft.ifftshift(torch.tensordot(torch.diag(fullmask).to(DTyp),y,dims=([1],[0])))
kplot(y,roll=False,log=True)
kplot(z,roll=True,log=True)

In [ ]:
imgHeg = 320; imgWid = 320
x_ifft = torch.abs(F.ifftn(z,dim=(0,1),norm='ortho')).to(dtyp)
x_unet = UNET(x_ifft.view(1,1,imgHeg,imgWid)).detach()

print('error of x_ifft = ', torch.norm(torch.flatten(x_ifft)-torch.flatten(xstar),'fro')/torch.norm(xstar,'fro') )
print('error of x_unet = ', torch.norm(torch.flatten(x_unet)-torch.flatten(xstar),'fro')/torch.norm(xstar,'fro') )

In [ ]:
from sigpy.mri.app import TotalVariationRecon
NN = 50
Lambda_grid = 10**np.linspace(-3.5,-3,NN)
errRec = np.zeros((NN))
ind = 1

mps  = np.ones((1,imgHeg,imgWid))
y_sp = np.reshape(z.numpy(),(-1,imgHeg,imgWid))
for Lambda in Lambda_grid:
# Lambda = 10**(-6.31) 
# Lambda = 10**(-3.2755) # np.log10(Lambda_grid[np.argmin(errRec)])
    print('{} out of {}'.format(ind,NN))
    x_sp = np.fft.fftshift( np.abs(TotalVariationRecon(y_sp, mps, Lambda,show_pbar=False).run()) ) 
    x_sp = torch.tensor(x_sp)
    errRec[ind-1] = torch.norm(torch.flatten(x_sp)-torch.flatten(xstar),'fro')/torch.norm(xstar,'fro')
    ind += 1
#     print('error of x_sp = ', torch.norm(torch.flatten(x_sp)-torch.flatten(xstar),'fro')/torch.norm(xstar,'fro'))

plt.figure()
plt.xscale('log')
plt.plot(Lambda_grid,errRec)

In [ ]:
np.linspace(-6,-4,gridnum)[8]

In [ ]:
reload(mask_backward_new)
from mask_backward_new import mask_backward,mask_eval

maxIter = 200
gridnum = 10
alpha_grid = 10**np.linspace(-4.6,-4.5,gridnum)
sr_rec = np.zeros((gridnum))
mloss_rec = np.zeros(gridnum)

ind = 0
for alpha in alpha_grid:
    print('[{}/{}]  alpha {}'.format(ind+1,gridnum,alpha))
    highmask_refined,refine_mloss,init_loss = mask_backward(highmask,xstar,\
                          beta=1., alpha=alpha,maxIter=maxIter,seed=0,break_limit=maxIter//2,\
                          lr=5e-4,mode='UNET',\
                          verbose=False,dtyp=dtyp)
    print('Difference between masks: \n',highmask_refined - highmask)
#     print('Refined mask is: \n',highmask_refined)
    mloss_rec[ind] = refine_mloss
    sr_rec[ind] = (torch.sum(highmask_refined).item() + 24)/320
    ind += 1

In [ ]:
plt.figure()
plt.scatter(alpha_grid,mloss_rec,label='refined')
plt.scatter(alpha_grid,init_loss*np.ones(mloss_rec.shape),label='init.')
plt.title('mask loss')
plt.xscale('log')
plt.legend(loc='best')
plt.show()

plt.figure()
plt.scatter(alpha_grid,sr_rec,label='refined')
plt.scatter(alpha_grid,0.25*np.ones(mloss_rec.shape),label='init.')
plt.title('mask sampling ratio')
plt.xscale('log')
plt.legend(loc='best')
plt.show()

### arXiv

In [ ]:
reload(mask_backward_new)
from mask_backward_new import mask_backward

# mode ADMM:

highmask = torch.zeros((round(320*0.9)))
highmask[np.random.choice(round(320*0.9),int(320*.1),replace=False)] = 1

naive_mloss = mask_eval(mask_complete(highmask.to(torch.double),320),xstar,unroll_block=unroll_block,Lambda=Lambda,rho=rho) * 100
highmask_refined,refine_mloss = mask_backward(highmask,xstar,\
                          beta=1., alpha=1e1,maxIter=200,unroll_block=unroll_block,seed=0,break_limit=100,\
                          lr=5e-4,mode='ADMM',Lambda=Lambda,rho=rho,\
                          verbose=True,perturb=False,perturb_freq=5,eps=1e-2)

In [ ]:
# vary sampling ratio, observe RMSE for the same image.
reload(mask_backward_new)
from mask_backward_new import mask_backward

base=0.1
r_grid = np.array([0.1,0.15,0.2])
naive_mloss = np.zeros((r_grid.size))
refine_mloss = np.zeros((r_grid.size))

sampRatio = np.zeros((r_grid.size))

unroll_block = 6; Lambda=6.1e-4; rho=1e1; lr = 5e-2
maxIter = 300; break_limit = round(maxIter/3)

np.random.seed(0)

ind = 0
for r in r_grid:
    print('\n r {}, the {} item out of {}'.format(r,ind+1,r_grid.size))
    highmask = torch.zeros((round(320*(1-base))))
    highmask[np.random.choice(round(320*(1-base)),int(320*r),replace=False)] = 1
    
    naive_mloss[ind] = mask_eval(mask_complete(highmask.to(torch.double),320),xstar,unroll_block=unroll_block,Lambda=Lambda,rho=rho) * 100
    highmask_refined,refine_mloss[ind] = mask_backward(highmask,xstar,seed=0,\
                              beta=1., alpha=9.5e0,maxIter=maxIter,unroll_block=unroll_block,break_limit=break_limit,\
                              lr=lr,mode='ADMM',Lambda=Lambda,rho=rho,\
                              verbose=True,perturb=False,perturb_freq=5,eps=1e-2)
    sampRatio[ind] = (highmask_refined.sum().item() + round(320*0.1))/320
    ind += 1

In [ ]:
print('sampRatio: ',sampRatio)
plt.figure()
plt.scatter(r_grid,naive_mloss,label='naive')
plt.scatter(sampRatio,refine_mloss,label='refined')
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left")

In [ ]:
## find a good alpha for l1 penalty
reload(mask_backward_new); reload(utils);
from mask_backward_new import mask_backward
alpha_grid = np.linspace(1,10,10)
sparsity_ = np.zeros(alpha_grid.size)
change_count = np.zeros(alpha_grid.size)

unroll_block = 6; Lambda=6.1e-4; rho=1e1; lr = 5e-2
maxIter = 300; break_limit = round(maxIter/2)
# add samp. ratio. = .1 ---> best alpha = ?
# add samp. ratio. = .2 ---> best alpha = ?
# add samp. ratio. = .2 ---> best alpha = ?

base = 0.05
additional = 0.125

highmask = torch.zeros((round(320*(1-base))))
highmask[np.random.choice(round(320*(1-base)),round(320*additional),replace=False)] = 1

ind = 0
for alpha in alpha_grid:
    print('\n\talpha = {}'.format(alpha))
    highmask_refined,_ = mask_backward(highmask,xstar,\
                  beta=1, alpha=alpha,maxIter=maxIter,unroll_block=unroll_block,seed=0,break_limit=break_limit,\
                  lr=lr,mode='ADMM',Lambda=Lambda,rho=rho,\
                  perturb=False,perturb_freq=10,verbose=True)
    sparsity_[ind] = mask_complete(highmask_refined.to(torch.double),320).sum().item()/320
    change_count[ind] = torch.abs(highmask_refined - highmask).sum().item()
    ind += 1

In [ ]:
plt.figure()
plt.scatter(alpha_grid,sparsity_,label='end')
plt.scatter(alpha_grid,(base+additional)*np.ones(alpha_grid.size),label='start')
plt.title('sparsity')
# plt.xscale('log')
plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left")

plt.figure()
plt.scatter(alpha_grid,change_count)
# plt.xscale('log')
plt.title('change count')

In [ ]:
naive_mloss = mask_eval(mask_complete(highmask,imgHeg,dtyp=dtyp),xstar,mode='UNET',UNET=UNET,dtyp=dtyp) * 100
print('naive mask loss = ',naive_mloss)

In [ ]:
### verify that UNET works fine
z = apply_mask(full_gredmask,yfull,mode='c')
xs = torch.abs(F.ifftn(F.ifftshift(z),dim=(1,2),norm='ortho'))
xs = torch.reshape(xs,(xs.shape[0],1,xs.shape[1],xs.shape[2]))

UNET.eval()
imgind = 0
xrecon = UNET(xs)
plt.figure()
plt.title('recon')
plt.imshow(xrecon.detach().numpy()[imgind,0,:,:])
plt.colorbar()
plt.show()

plt.figure()
plt.title('gt')
plt.imshow(xstar.detach().numpy()[imgind,:,:])
plt.colorbar()
plt.show()
print(torch.norm(xrecon[imgind,0,:,:]-xstar[imgind,:,:],'fro')/torch.norm(xstar[imgind,:,:],'fro').item())

In [ ]:
reload(utils)
from utils import hfen

In [ ]:
torch.squeeze(xrecon)[0,:,:].shape

In [ ]:
hfen(torch.squeeze(xrecon)[imgind,:,:],xstar[imgind,:,:],base=0)